# 0. Load Libraries

In [1]:
import numpy as np
import os
import json
import pandas as pd
import models
import utils
from classes import  Locatable, Satellite, Cluster, Vehicle
from drawingmap import DrawingMap

# 1. Load Data

## 1.1. Satellites

In [2]:
satellites, df_satellites = utils.LoadingData.load_satellites(DEBUG=False)
print(f'Cantidad de satellites cargados: {len(satellites)}')

Cantidad de satellites cargados: 9


## 1.2. Customer Clusters

In [3]:
clusters, df_clusters = utils.LoadingData.load_customer_clusters(DEBUG=False)
print(f'Cantidad de clusters cargados: {len(clusters)}')

Cantidad de clusters cargados: 721


## 1.3. Load Vehicles

In [4]:
small_vehicle = Vehicle(id='small'
                        , type='small'
                        , capacity=12
                        , costFixed=20
                        , time_service=0.05
                        , time_fixed=0.05
                        , time_load=0.0072
                        , time_dispatch=0.625
                        , speed_line=40
                        , Tmax=12
                        , k=1.3)
large_vehicle = Vehicle(id='large'
                        , type='large'
                        , capacity=53
                        , costFixed=268
                        , time_service=0.05
                        , time_fixed=0.05
                        , time_load=0.0142
                        , time_dispatch=0.75
                        , speed_line=80
                        , Tmax=12
                        , k=1.3)

## 1.4. Load Matrixes: Distance and Durations with and without traffic

### 1.4.1. From Satellites to Clusters

In [5]:
matrixes_from_satellites = utils.LoadingData.load_distances_duration_matrix_from_satellite()

### 1.4.2. From DC to Clusters

In [6]:
matrixes_from_dc = utils.LoadingData.load_distances_duration_matrix_from_dc()

# 2. Drawing Map

## 2.1. La Paz with customer segments

In [7]:
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)

drawer = DrawingMap(location_la_paz)

drawer.addNodes(list_locatables=list(clusters.values()), color="blue", radius=1)
drawer.addNodes(list_locatables=list(satellites.values()), color='red', radius=3)
drawer.addMarker(location=location_DC, label='DC')

map = drawer.viewMap()
map

# 3. Generate Params

In [8]:
periods = 12

params_gurobi = {
    'TimeLimit':3600,
    'MIPGap':0.05
}

In [9]:
satellite_prueba = satellites['Abaroa']

In [10]:
cluster_prueba = clusters['89b3219820bffff']

## 3.1. Average Fleet Size
#### a) From Satellite to Customer Cluster

In [11]:
config_ = utils.ConfigDeterministic()
fleet_size_satellites = config_.calculate_avg_fleet_size_from_satellites(satellites=list(satellites.values())
                                                                         , clusters=list(clusters.values())
                                                                         , vehicle=small_vehicle
                                                                         ,periods=periods, distances_linehaul=matrixes_from_satellites['distance'])

#### b) From DC to Customer Cluster

In [12]:
fleet_size_dc = config_.calculate_avg_fleet_size_from_dc(clusters=clusters.values(), vehicle=large_vehicle, periods=periods
                                                         , distances_linehaul=matrixes_from_dc['distance'])

In [13]:
fleet_size_required = {
    'small': fleet_size_satellites,
    'large': fleet_size_dc
}

## 3.3. Cost *$c_{sk}^{t}$* and *$g_{k}{t}$*

In [14]:
distance_average_from_satellites = {}
for s in satellites.values():
    sum_distances = 0
    count_records = 0
    for k in clusters.values():
        sum_distances+=matrixes_from_satellites['distance'][(s.id, k.id)]
        count_records=count_records+1
    avg_distance = sum_distances/count_records
    distance_average_from_satellites[s.id] = avg_distance
distance_average_from_satellites

{'Abaroa': 7.163305131761442,
 'Llojeta': 9.200861303744794,
 'Cota Cota': 10.82359778085992,
 'Achachicala': 8.87314285714286,
 'Mallasa': 12.356907073509012,
 'Sopocachi': 7.580083217753125,
 'Perferica': 8.806714285714287,
 'Zona Cementerio': 9.16176976421637,
 'Los Pinos': 8.961859916782249}

In [15]:
distance_average_from_dc = 0
sum_distances = 0
count_records = 0
for k in clusters.values():
    sum_distances+=matrixes_from_dc['distance'][k.id]
    count_records=count_records+1
avg_distance = sum_distances/count_records
distance_average_from_dc = avg_distance
distance_average_from_dc

18.852235783633862

In [16]:
# cost of shipping
fee_cost_from_satellites = 0.071 #0.071
fee_cost_from_dc = 0.06 #0.06 #

cost_shipping_from_satellites = dict([
    ((s.id, k.id),fee_cost_from_satellites*matrixes_from_satellites['distance'][(s.id, k.id)]/distance_average_from_satellites[s.id])
    if matrixes_from_satellites['distance'][(s.id, k.id)]/distance_average_from_satellites[s.id] > 0.5 else ((s.id, k.id),fee_cost_from_satellites*0.5)
    for s in satellites.values() for k in clusters.values()
])


cost_shipping_from_dc = dict([
    ((k.id),fee_cost_from_dc*matrixes_from_dc['distance'][k.id]/distance_average_from_dc)
    if matrixes_from_dc['distance'][k.id]/distance_average_from_dc> 0.5 else ((k.id),fee_cost_from_dc*0.5)
    for k in clusters.values()
])

In [17]:
def cost_satellite_cluster_period(satellites: list[Satellite],
                                  clusters: list[Cluster],
                                  cost_shipping: dict[(str, str), float],
                                  vehicle_small: Vehicle,
                                  periods: int,
                                  vehicles_required: dict[str, dict]) -> dict[(str,str,int), float]:
    costs = {}
    for t in range(periods):
        for k in clusters:
            for s in satellites:
                costs[(s.id, k.id, t)] = (s.costSourcing*k.demandByPeriod[t]) + (cost_shipping[(s.id, k.id)]*k.demandByPeriod[t]) + \
                                         (vehicle_small.costFixed*vehicles_required['small'][(s.id, k.id, t)])
    return costs

def cost_dc_cluster_period(clusters: list[Cluster],
                           cost_shipping: dict[str, float],
                           vehicle_large: Vehicle,
                           periods: int,
                           vehicles_required: dict[str, dict]) -> dict[(str,int), float]:
    costs = {}
    for t in range(periods):
        for k in clusters:
            costs[(k.id, t)] = (cost_shipping[k.id] * k.demandByPeriod[t]) + (vehicle_large.costFixed * vehicles_required['large'][(k.id, t)])
    return costs

In [18]:
cost_C = cost_satellite_cluster_period(satellites.values(), clusters.values()
                                       ,cost_shipping=cost_shipping_from_satellites
                                       ,vehicle_small=small_vehicle
                                       ,periods=periods
                                       ,vehicles_required=fleet_size_required)

cost_G = cost_dc_cluster_period(clusters.values()
                                ,cost_shipping=cost_shipping_from_dc
                                ,vehicle_large=large_vehicle
                                ,periods=periods
                                ,vehicles_required=fleet_size_required)
cost_operation = {
    'satellite': cost_C,
    'dc': cost_G
}

# 4. Model Deterministic

In [19]:
model_ = models.ModelDeterministic(periods=periods)
model_.setParams(params=params_gurobi)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-13
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.05


In [20]:
model_.build(satellites=list(satellites.values())
             ,clusters=list(clusters.values())
             ,vehicles_required=fleet_size_required
             ,costs=cost_operation)

Discarded solution information


{'time_building': 1}

In [21]:
print(model_.optimizeModel())

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 86745 rows, 86682 columns and 319404 nonzeros
Model fingerprint: 0x1af4d4a6
Variable types: 0 continuous, 86682 integer (86682 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+01]
  Objective range  [6e-02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 30368.573328
Presolve removed 2427 rows and 10843 columns
Presolve time: 0.31s
Presolved: 84318 rows, 75839 columns, 304040 nonzeros
Variable types: 0 continuous, 75839 integer (75839 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   82001    1.7671702e+04   0.000000e+00   8.417909e+04      5s
   85961    1.7638868e+04   0.000000e+00   9.374

In [22]:
modelito = model_.model

In [27]:
cost_allocation_satellites = np.sum([
    (s.costFixed[q_id]/10) * model_.Y[(s.id, q_id)].x for s in list(satellites.values()) for q_id in s.capacity.keys()
])

cost_operating_satellites = np.sum([
    (s.costOperation[t]/10) * model_.X[(s.id, t)].x for s in list(satellites.values()) for t in range(periods)
])

cost_served_from_satellite = np.sum([
    cost_operation['satellite'][(s.id, k.id, t)] * model_.Z[(s.id, k.id, t)].x for s in list(satellites.values()) for k in
    list(clusters.values()) for t in
    range(periods)
])

cost_served_from_dc = np.sum([
    cost_operation['dc'][(k.id, t)] * model_.W[(k.id, t)].x for k in list(clusters.values()) for t in range(periods)
])

In [28]:
cost_allocation_satellites

1200.0

In [29]:
cost_operating_satellites

3190.7999999999997

In [30]:
cost_served_from_satellite

12359.500498325131

In [31]:
cost_served_from_dc

908.219683478567

## 4.1. Get results

In [35]:
variables_results = model_.get_results(satellites=list(satellites.values())
                                      ,clusters=list(clusters.values()))

### 4.1.1. Y

In [49]:
for k, v in variables_results['Y'].items():
    print(f'id: {v.id} cap: {k[1]}')

id: Abaroa cap: 10
id: Los Pinos cap: 3


### 4.1.2. X

In [53]:
for t in range(periods):
    print(f'###### t - {t}', end=" ")
    for k, v in variables_results['X'][t].items():
        print(f'id: {v.id}', end=" | ")
    print("")

###### t - 0 id: Abaroa | 
###### t - 1 id: Abaroa | id: Los Pinos | 
###### t - 2 id: Abaroa | 
###### t - 3 id: Abaroa | id: Los Pinos | 
###### t - 4 id: Abaroa | 
###### t - 5 id: Abaroa | 
###### t - 6 id: Abaroa | id: Los Pinos | 
###### t - 7 id: Abaroa | 
###### t - 8 id: Abaroa | id: Los Pinos | 
###### t - 9 id: Abaroa | id: Los Pinos | 
###### t - 10 id: Abaroa | id: Los Pinos | 
###### t - 11 id: Abaroa | id: Los Pinos | 


### 4.1.3. W

In [85]:
period_selected = 1
lista_clusters = variables_results['W'][period_selected]

# map
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)
drawer = DrawingMap(location_la_paz)
drawer.addNodes(list_locatables=lista_clusters, color="blue", radius=3)
drawer.addMarker(location=location_DC, label='DC')
map = drawer.viewMap()
map

### 4.1.4 Z

In [86]:
map_colors = {
    'Abaroa': 'blue',
    'Llojeta': 'orange',
    'Cota Cota': 'green',
    'Achachicala': 'darkgreen',
    'Mallasa': 'black',
    'Sopocachi': 'gray',
    'Perferica': 'darkred',
    'Zona Cementerio': 'pink',
    'Los Pinos': 'purple'
}

In [87]:
period_selected = 1
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)
drawer = DrawingMap(location_la_paz)
for k, v in map_colors.items():
    satellite_selected = k
    lista_clusters = variables_results['Z'][period_selected][satellite_selected]
    if len(lista_clusters) == 0:
        continue
    # map
    drawer.addNodes(list_locatables=lista_clusters, color=v, radius=1)
    drawer.addNode(location=(satellites[satellite_selected].lat, satellites[satellite_selected].lon), color=v, radius=4)

drawer.addMarker(location=location_DC, label='DC')
map = drawer.viewMap()
map

In [74]:
period_selected = 1
satellite_selected = "Abaroa"
lista_clusters = variables_results['Z'][period_selected][satellite_selected]
# map
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)
drawer = DrawingMap(location_la_paz)
drawer.addNodes(list_locatables=lista_clusters, color="blue", radius=1)
drawer.addNode(location=(satellites[satellite_selected].lat, satellites[satellite_selected].lon), color='red', radius=3)
drawer.addMarker(location=location_DC, label='DC')

map = drawer.viewMap()
map